### Delegation 

In [ ]:
import pandas as pd

def delegate_op_tokens(data):
    """
    Function to delegate OP tokens from one delegate to another and update voting powers.

    Parameters:
        data (pd.DataFrame): The dataset containing delegate information and voting power.

    Returns:
        pd.DataFrame: Updated DataFrame with the modified voting powers.
    """

    # Prompt the user for delegate addresses
    from_delegate = input("Enter the address of the delegate who is delegating tokens: ")
    to_delegate = input("Enter the address of the delegate who will receive tokens: ")

    # Check if both delegates are in the data
    if from_delegate not in data['delegate'].values:
        print(f"Delegate {from_delegate} not found.")
        return data
    if to_delegate not in data['delegate'].values:
        print(f"Delegate {to_delegate} not found.")
        return data
    
    # Prompt thr user for the amount of OP tokens to delegate
    try:
        amount = float(input("Enter the amount of OP tokens to delegate: "))
        if amount <= 0:
            print("Amount should be a positive number.")
            return data
    except ValueError:
        print("Invalid input. Please enter a numeric value for the amount.")
        return data

    # Find the index of both delegates
    from_index = data[data['delegate'] == from_delegate].index[0]
    to_index = data[data['delegate'] == to_delegate].index[0]

    # Check if 'from_delegate' has enough voting power
    if data.loc[from_index, 'voting_power'] < amount:
        print(f"Not enough voting power to delegate. {from_delegate} only has {data.loc[from_index, 'voting_power']} OP tokens.")
        return data

    # Deduct the voting power from the 'from_delegate'
    data.loc[from_index, 'voting_power'] -= amount

    # Add the voting power to the 'to_delegate'
    data.loc[to_index, 'voting_power'] += amount

    # Update token house voting share 'th_vp' proportionally for each delegate
    total_voting_power = data['voting_power'].sum()
    data['th_vp'] = (data['voting_power'] * 100) / total_voting_power

    # Set 'th_vp' to 0 if voting_power of any delegate is less than 1
    data.loc[data['voting_power'] < 1, 'th_vp'] = 0

    print(f"Delegated {amount} OP tokens from {from_delegate} to {to_delegate}.")
    return data

# Load data
data = pd.read_csv("../Data/Test_Delegation_Functionality/2024-10-22.csv")

# Execute the delegation function
updated_data = delegate_op_tokens(data)

sorted_data = updated_data.sort_values(by='voting_power', ascending=False).reset_index(drop=True)

# Save the updated data to the file
sorted_data.to_csv("../Data/Test_Delegation_Functionality/2024-10-22.csv", index=False)
print("Data updated and saved.")


### Delegation with HHI and CPI Calculation

In [ ]:
import pandas as pd
from datetime import datetime

# Define each influence period with start_date, end_date, and influence percentages
influence_periods = [
    {
        "start_date": "2022-05-26", "end_date": "2023-01-25",
        "influences": {"th_vp": 48.32, "ch_vp_r2": 51.68}
    },
    {
        "start_date": "2023-01-26", "end_date": "2023-06-07",
        "influences": {"th_vp": 41.95, "ch_vp_r2": 44.88, "gc_vp_s3": 13.17}
    },
    {
        "start_date": "2023-06-08", "end_date": "2023-10-13",
        "influences": {"th_vp": 41.95, "ch_vp_r2": 44.88, "gc_vp_s4": 13.17}
    },
    {
        "start_date": "2023-10-14", "end_date": "2024-01-03",
        "influences": {"th_vp": 41.95, "ch_vp_r3": 44.88, "gc_vp_s4": 13.17}
    },
    {
        "start_date": "2024-01-04", "end_date": "2024-06-02",
        "influences": {
            "th_vp": 32.33, "ch_vp_r3": 34.59, "gc_vp_s5": 10.15,
            "gc_vp_mm_s5": 2.82, "sc_vp_s5": 12.78, "coc_vp_s5": 4.32, "dab_vp_s5": 3.01
        }
    },
    {
        "start_date": "2024-06-03", "end_date": "2024-06-26",
        "influences": {
            "th_vp": 32.33, "ch_vp_r4": 34.59, "gc_vp_s5": 10.15,
            "gc_vp_mm_s5": 2.82, "sc_vp_s5": 12.78, "coc_vp_s5": 4.32, "dab_vp_s5": 3.01
        }
    },
    {
        "start_date": "2024-06-27", "end_date": "2024-12-11",
        "influences": {
            "th_vp": 32.33, "ch_vp_r4": 34.59, "gc_vp_s6": 10.15,
            "gc_vp_mm_s6": 2.82, "sc_vp_s6": 12.78, "coc_vp_s6": 4.32, "dab_vp_s6": 3.01
        }
    }
]

def calculate_influence(row, influences):
    """Calculates influence based on influence percentages per column."""
    influence_sum = sum(row.get(col, 0) * (val / 100) for col, val in influences.items())
    return influence_sum

def add_influence_column(df, file_date_str):
    """Adds 'influence' column to DataFrame based on file date."""
    file_date = datetime.strptime(file_date_str, "%Y-%m-%d")
    for period in influence_periods:
        start_date = datetime.strptime(period["start_date"], "%Y-%m-%d")
        end_date = datetime.strptime(period["end_date"], "%Y-%m-%d")
        if start_date <= file_date <= end_date:
            df["influence"] = df.apply(calculate_influence, axis=1, influences=period["influences"])
            break
    return df

def calculate_HHI_and_CPI(data, file_date_str):
    """Calculate HHI and CPI based on the data."""
    data = add_influence_column(data, file_date_str)
    data['th_vp_squared'] = data['th_vp'] ** 2
    data['influence_squared'] = data['influence'] ** 2
    HHI = round(data['th_vp_squared'].sum(), 2)
    CPI = round(data['influence_squared'].sum(), 2)

    print(f"Date: {file_date_str} | HHI: {HHI} | CPI: {CPI}")
    return HHI, CPI

def delegate_op_tokens(data):
    """Delegates OP tokens from one delegate to another, updates 'th_vp', and recalculates HHI and CPI."""
    from_delegate = input("Enter address of delegate delegating tokens: ")
    to_delegate = input("Enter address of delegate receiving tokens: ")

    if from_delegate not in data['delegate'].values:
        print(f"Delegate {from_delegate} not found.")
        return data
    if to_delegate not in data['delegate'].values:
        print(f"Delegate {to_delegate} not found.")
        return data

    try:
        amount = float(input("Enter amount of OP tokens to delegate: "))
        if amount <= 0:
            print("Amount should be positive.")
            return data
    except ValueError:
        print("Invalid input for amount.")
        return data

    from_index = data[data['delegate'] == from_delegate].index[0]
    to_index = data[data['delegate'] == to_delegate].index[0]

    if data.loc[from_index, 'voting_power'] < amount:
        print(f"Insufficient voting power. {from_delegate} has {data.loc[from_index, 'voting_power']} OP tokens.")
        return data

    data.loc[from_index, 'voting_power'] -= amount
    data.loc[to_index, 'voting_power'] += amount

    # Update 'th_vp' and set to 0 for 'voting_power' < 1
    total_voting_power = data['voting_power'].sum()
    data['th_vp'] = (data['voting_power'] * 100) / total_voting_power
    data.loc[data['voting_power'] < 1, 'th_vp'] = 0

    print(f"Delegated {amount} OP tokens from {from_delegate} to {to_delegate}.")

    sorted_data = data.sort_values(by='voting_power', ascending=False).reset_index(drop=True)

    # Recalculate HHI and CPI
    data = add_influence_column(sorted_data, file_date_str)
    HHI, CPI = calculate_HHI_and_CPI(data, file_date_str)

    sorted_data.to_csv(file_path, index=False)
    return sorted_data

# Example usage:
file_path = "../Data/Test_Delegation_Functionality/2024-10-22.csv"
file_date_str = "2024-10-22"

# Load data
data = pd.read_csv(file_path)

# Initial Calculation of HHI and CPI
print("Initial HHI and CPI Calculation:")
calculate_HHI_and_CPI(data, file_date_str)

# Delegate tokens and update data with recalculated HHI and CPI
updated_data = delegate_op_tokens(data)

Initial HHI and CPI Calculation:
Date: 2024-10-22 | HHI: 316.34 | CPI: 72.43
Delegated 2000000.0 OP tokens from 0x3eee61b92c36e97be6319bf9096a1ac3c04a1466 to 0xf11b6a8c3cb8bb7dbc1518a613b10ceb0bbfc06b.
Date: 2024-10-22 | HHI: 302.46 | CPI: 70.98
